In [49]:
from capo.analysis.utils import get_results, aggregate_results
import pandas as pd

In [50]:
models = ["llama", "qwen", "mistral"]
datasets = ["sst-5", "agnews", "copa", "gsm8k"]
optims = ["CAPO", "EvoPromptGA", "PromptWizard", "OPRO"]

ablations = [
    "CAPO_no_lp",
    "CAPO_generic_init",
    "CAPO_no_racing",
    "CAPO_no_shuffling",
    "CAPO_zero_shot",
    "EvoPromptGA_generic_init",
    "EvoPromptGA_TD",
    "CAPO_gamma_0.1",
    "CAPO_gamma_0.01",
    "CAPO_gamma_0.02",
    "CAPO_gamma_0.05",
    "CAPO_ncrossovers_4",
    "CAPO_ncrossovers_7",
    "CAPO_ncrossovers_10",
    "CAPO_pop_8",
    "CAPO_pop_10",
    "CAPO_pop_12",
]

In [53]:
dfs = []

for model in models:
    for dataset in datasets:
        for optim in optims:
            df = get_results(dataset, model, optim, path_prefix="..")
            dfs.append(df.assign(model=model, dataset=dataset, optim=optim))

for ablation in ablations:
    for dataset in ["agnews", "gsm8k"]:
        df = get_results(dataset, "llama", ablation, path_prefix="..")
        dfs.append(df.assign(model=model, dataset=dataset, optim=ablation))

df = pd.concat(dfs, ignore_index=True)

In [54]:
# find first steps
df_min = df.groupby(["model", "dataset", "optim", "seed"]).first()["timestamp"]
df_min = pd.to_datetime(df_min)
df_max = df.groupby(["model", "dataset", "optim", "seed"]).last()["timestamp"]
df_max = pd.to_datetime(df_max)

In [55]:
df = df_max - df_min

In [62]:
pd.set_option("display.max_rows", None)

print(df)

model    dataset  optim                     seed
llama    agnews   CAPO                      42     0 days 02:12:29.287608
                                            43     0 days 01:54:04.773118
                                            44     0 days 02:49:39.144641
                  EvoPromptGA               42     0 days 01:23:18.260005
                                            43     0 days 01:42:41.846304
                                            44     0 days 01:35:32.547111
                  OPRO                      42     0 days 01:49:53.886359
                                            43     0 days 01:36:50.175038
                                            44     0 days 01:30:58.049324
                  PromptWizard              42                        NaT
                                            43                        NaT
                                            44                        NaT
         copa     CAPO                      42     0 days 03:13

In [56]:
df.sum()

Timedelta('13 days 02:36:55.033814')